Notebook showing off basics of node -> edge representation conversion.

In [1]:
import networkx as nx
from collections import OrderedDict
from itertools import chain

from IPython.display import IFrame

In [50]:
class OrderedMultiDiGraph(nx.MultiDiGraph):
    node_dict_factory = OrderedDict
    adjlist_dict_factory = OrderedDict
    edge_key_dict_factory = OrderedDict
    
    def add_edge(self, u, v, **kwargs):
        u.children.append(v)
        v.parents.append(u)
        return super(self.__class__, self).add_edge(u, v, **kwargs)

    def add_edges_from(self, edges_list, **kwargs):
        for edge in edges_list:
            if len(edge) == 2:
                u, v = edge
                self.add_edge(u, v, **kwargs)
            elif len(edge) == 3:
                u, v, d = edge
                self.add_edge(u, v, attr_dict=d, **kwargs)


In [51]:
class Particle(object):
    
    def __init__(self, name):
        self.name  = name
        self.children = []
        self.parents = []

    def __repr__(self):
        return "%s(children=%s, parents=%s)" % (self.name, 
                                                ', '.join([c.name for c in self.children]), 
                                                ', '.join([p.name for p in self.parents]))
    
    def __str__(self):
        return "%s(children=%s, parents=%s)" % (self.name, 
                                                ', '.join([c.name for c in self.children]), 
                                                ', '.join([p.name for p in self.parents]))
    
    def __eq__(self, other):
        return self.name == other.name

In [52]:
def node_graph_to_graphviz(gr):
    s = 'digraph g {\n'
    s += '    rankdir="LR";\n'
    for edge in gr.edges():
        s += '    %s -> %s;\n' % (edge[0].name, edge[1].name)
    s += '\n}'
    return s

def write_node_graph_to_file(gr, filename):
    with open(filename, 'w') as f:
        f.write(node_graph_to_graphviz(gr))

In [53]:
A = Particle("A")
B = Particle("B")
C = Particle("C")
D = Particle("D")
E = Particle("E")
F = Particle("F")
G = Particle("G")
H = Particle("H")
I = Particle("I")

# gr = nx.MultiDiGraph()
gr = OrderedMultiDiGraph()

gr.add_edges_from([(A, C), (A, D), (B, D), (B, E)]) # VBF diagram

# gr.add_edges_from([(A, D), (B, D), (C, D), (B, E), (C, E)]) # scenario 3

# gr.add_edges_from([(A, C), (A, D), (B, C), (B, D)]) # 4 body vertex

# gr.add_edges_from([(A, C), (A, D), (B, D)]) # scenario 1

# gr.add_edges_from([(A, E), (B, E), (B, F), (C, F), (C, G), (D, G)])

In [54]:
gr.nodes()

[A(children=C, D, parents=),
 C(children=, parents=A),
 D(children=, parents=A, B),
 B(children=D, E, parents=),
 E(children=, parents=B)]

In [55]:
gr.edges()

[(A(children=C, D, parents=), C(children=, parents=A)),
 (A(children=C, D, parents=), D(children=, parents=A, B)),
 (B(children=D, E, parents=), D(children=, parents=A, B)),
 (B(children=D, E, parents=), E(children=, parents=B))]

In [56]:
filename_stem = 'node_orig'
write_node_graph_to_file(gr, filename_stem+'.gv')

!dot -Tpdf "$filename_stem".gv -o "$filename_stem".pdf

IFrame(filename_stem+'.pdf', width=600, height=400)

In [57]:
# consider a node
node = gr.nodes()[0]
print "This node:", node

# construct a sub-graph of all children & parents
if len(node.children) > 0:
    total_parents = []
    total_children = node.children[:]

    print "Starting Total parents:", total_parents
    print "Starting Total children:", total_children
    
    def all_parents(children):
        return list(set([p for c in children for p in c.parents]))

    def all_children(parents):
        return list(set([c for p in parents for c in p.children]))

    # iterative algo to get all parents an children
    while (set(total_parents) != set(all_parents(total_children)) 
           or set(total_children) != set(all_children(total_parents))):

        total_parents.extend(list(set(all_parents(total_children)).difference(set(total_parents))))
        total_children.extend(list(set(all_children(total_parents)).difference(set(total_children))))

    print "Final Total parents:", total_parents
    print "Final Total children:", total_children

    # now decide if duplication required
    do_duplicate = False
    for ch in total_children:
        if len(ch.parents) != len(total_parents):
            do_duplicate = True
            print "Need duplicate as", ch, "has", len(ch.parents), "/", len(total_parents), "parents of this subgraph"
            print "Parents:", ch.parents

    # if so, add in duplicate node(s), ensuring connections correct
    
    if do_duplicate:
        # duplicate graph to avoid infinite loops...
        gr_dupl = gr.copy()
        print 'Starting with', gr_dupl.edges()
        for pa in total_parents:
            if len(pa.children) > 1:
                for ch in pa.children:
                    if len(ch.parents) > 1:
                        print "Removing", pa.name, ch.name
                        ed = [e for e in gr_dupl.edges() if e[0] == pa and e[1] == ch]
                        try:
                            ed = ed[0]
                        except IndexError as e:
                            print 'Cannot get edge', pa, ch
                            raise
                        print ed
                        gr_dupl.remove_edge(*ed)
                        new_name = pa.name + "_" + ch.name
                        dupl = Particle(new_name)
                        print 'Adding', dupl
                        # need the duplicate objects
                        pa_dupl = [n for n in gr_dupl.nodes() if n == pa][0]
                        ch_dupl = [n for n in gr_dupl.nodes() if n == ch][0]
                        ch_dupl.parents.remove(pa)
                        pa_dupl.children.remove(ch_dupl)
                        gr_dupl.add_edge(pa_dupl, dupl)
                        gr_dupl.add_edge(dupl, ch_dupl)
        
        gr = gr_dupl

This node: A(children=C, D, parents=)
Starting Total parents: []
Starting Total children: [C(children=, parents=A), D(children=, parents=A, B)]
Final Total parents: [A(children=C, D, parents=), B(children=D, E, parents=)]
Final Total children: [C(children=, parents=A), D(children=, parents=A, B), E(children=, parents=B)]
Need duplicate as C(children=, parents=A) has 1 / 2 parents of this subgraph
Parents: [A(children=C, D, parents=)]
Need duplicate as E(children=, parents=B) has 1 / 2 parents of this subgraph
Parents: [B(children=D, E, parents=)]
Starting with [(A(children=C, D, parents=), C(children=, parents=A)), (A(children=C, D, parents=), D(children=, parents=A, B)), (B(children=D, E, parents=), D(children=, parents=A, B)), (B(children=D, E, parents=), E(children=, parents=B))]
Removing A D
(A(children=C, D, parents=), D(children=, parents=A, B))
Adding A_D(children=, parents=)
Removing B D
(B(children=D, E, parents=), D(children=, parents=B, A_D))
Adding B_D(children=, parents=)


In [58]:
print gr.nodes()
print gr.edges()

[A(children=C, A_D, parents=), C(children=, parents=A), D(children=, parents=A_D, B_D), B(children=E, B_D, parents=), E(children=, parents=B), A_D(children=D, parents=A), B_D(children=D, parents=B)]
[(A(children=C, A_D, parents=), C(children=, parents=A)), (A(children=C, A_D, parents=), A_D(children=D, parents=A)), (B(children=E, B_D, parents=), E(children=, parents=B)), (B(children=E, B_D, parents=), B_D(children=D, parents=B)), (A_D(children=D, parents=A), D(children=, parents=A_D, B_D)), (B_D(children=D, parents=B), D(children=, parents=A_D, B_D))]


In [59]:
filename_stem = 'node'
write_node_graph_to_file(gr, filename_stem+'.gv')

!dot -Tpdf "$filename_stem".gv -o "$filename_stem".pdf

IFrame(filename_stem+'.pdf', width=600, height=400)

In [60]:
def edge_graph_to_graphviz(gr):
    s = 'digraph g {\n'
    s += '    rankdir="LR";\n'
    s += '    node[shape="point"];\n'
    for edge in gr.edges():
        u, v = edge
        d = gr.edge[u][v][0]
        s += '    %s -> %s[label="%s"];\n' % (u, v, d['particle'].name)
    s += '\n}'
    return s

def write_edge_graph_to_file(gr, filename):
    with open(filename, 'w') as f:
        f.write(edge_graph_to_graphviz(gr))

In [61]:
# convert to edge graph...
gr_new = nx.MultiDiGraph()

edge_list = []

# for each particle, add an edge
for particle in gr.nodes():
    u = len(edge_list) * 2
    v = u + 1
    attr = dict(particle=particle)
    p_edge = (u, v, attr)
    edge_list.append(p_edge)
    
# print edge_list 

gr_new.add_edges_from(edge_list)

# gr_new.edge

In [62]:
def get_edge(gr, particle):
    for (u, v) in gr.edges():
#         print u, v
        if len(gr.edge[u][v])>1:
            print gr.edge[u][v]
        if gr.edge[u][v][0]['particle'] == particle:  # [0] for Multi graph...
            return u, v
        
# now simplify vertices for parent/children relationships
for particle in gr.nodes():
    out_vtx, in_vtx = get_edge(gr_new, particle)
    print "****", particle, out_vtx, in_vtx
 
    total_parents = []
    total_children = particle.children[:]

#     print "Starting Total parents:", total_parents
#     print "Starting Total children:", total_children
    
    def all_parents(children):
        parents = list(set([p for c in children for p in c.parents]))
#         print 'all_parents (children):', children
#         print 'all_parents (parents):', parents
        return parents

    def all_children(parents):
        children = list(set([c for p in parents for c in p.children]))
#         print 'all_children (parents):', parents
#         print 'all_children (children):', children
        return children

    # iterative algo to get all parents an children
    while (set(total_parents) != set(all_parents(total_children)) 
           or set(total_children) != set(all_children(total_parents))):

        total_parents.extend(list(set(all_parents(total_children)).difference(set(total_parents))))
        total_children.extend(list(set(all_children(total_parents)).difference(set(total_children))))
#         print 'total_parents:', total_parents
#         print 'total_children:', total_children
        
#     print 'Final', total_parents
#     print 'Final', total_children

    for child in total_children:
#         print '\tDoing', parent
        o, v = get_edge(gr_new, child)
#         print '\tDoing', child, o, v
        if o == in_vtx:
            continue
        e_dict = gr_new.edge[o][v].values()[0]  # what if multiple versions???
#         print '\tRemoving edge', o, v
        gr_new.remove_edge(o, v)
        if gr_new.edge[o] == {}:
#             print '\tRemoving node', o
            gr_new.remove_node(o)
#         print '\tAdding new edge', in_vtx, v, e_dict
        gr_new.add_edge(in_vtx, v, attr_dict=e_dict)
#         print gr_new.edge
        
    for parent in total_parents:
        if parent == particle:
            continue
#         print '\tDoing', parent
        o, v = get_edge(gr_new, parent)
#         print '\tDoing', parent, o, v
        if v == in_vtx:
            continue
        e_dict = gr_new.edge[o][v].values()[0]  # what if multiple versions???
#         print '\tRemoving edge', o, v
        gr_new.remove_edge(o, v)
        if gr_new.edge[o] == {}:
#             print '\tRemoving node', o
            gr_new.remove_node(v)
#         print '\tAdding new edge', o, in_vtx, e_dict
        gr_new.add_edge(o, in_vtx, attr_dict=e_dict)
#         print gr_new.edge

**** A(children=C, A_D, parents=) 0 1
**** C(children=, parents=A) 1 3
**** D(children=, parents=A_D, B_D) 4 5
**** B(children=E, B_D, parents=) 6 7
**** E(children=, parents=B) 7 9
**** A_D(children=D, parents=A) 1 11
**** B_D(children=D, parents=B) 7 11


In [63]:
filename_stem = 'edge'
write_edge_graph_to_file(gr_new, filename_stem+'.gv')

!dot -Tpdf "$filename_stem".gv -o "$filename_stem".pdf

IFrame(filename_stem+'.pdf', width=600, height=400)